## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-22-20-57-02 +0000,nypost,DNA in Nancy Guthrie case could take ‘a year’ ...,https://nypost.com/2026/02/22/us-news/dna-in-n...
1,2026-02-22-20-56-59 +0000,bbc,Mexico's most wanted drug lord 'El Mencho' kil...,https://www.bbc.com/news/articles/cy4wywnrdd8o...
2,2026-02-22-20-56-54 +0000,nyt,"When Faced With Claims of Racism, Trump Points...",https://www.nytimes.com/2026/02/22/us/politics...
3,2026-02-22-20-55-45 +0000,nyt,Canada’s Spirit Rises and Falls With Its Olymp...,https://www.nytimes.com/2026/02/22/world/canad...
4,2026-02-22-20-55-30 +0000,nyt,Iran Students Protest for Second Day Despite S...,https://www.nytimes.com/2026/02/22/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2533/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,39
37,iran,11
19,killed,9
71,new,9
143,tariff,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
85,2026-02-22-14-09-27 +0000,cbc,"Armed man shot, killed after entering perimete...",https://www.cbc.ca/news/world/trump-mar-a-lago...,89
36,2026-02-22-19-30-35 +0000,bbc,Armed man killed after entering secure perimet...,https://www.bbc.com/news/articles/cp32l9w9pgyo...,77
174,2026-02-21-23-57-00 +0000,wsj,President Trump announced a 15% global tariff ...,https://www.wsj.com/economy/trade/what-to-know...,74
136,2026-02-22-08-08-24 +0000,bbc,Trump says he will increase his new global tar...,https://www.bbc.com/news/articles/cn8z48xwqn3o...,70
140,2026-02-22-06-32-45 +0000,nypost,Trump reveals his ‘new hero’ Supreme Court jus...,https://nypost.com/2026/02/22/us-news/trump-ca...,70


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
85,89,2026-02-22-14-09-27 +0000,cbc,"Armed man shot, killed after entering perimete...",https://www.cbc.ca/news/world/trump-mar-a-lago...
146,41,2026-02-22-04-47-09 +0000,nypost,Iran president says Tehran won’t ‘bow’ as war ...,https://nypost.com/2026/02/21/world-news/iran-...
45,31,2026-02-22-18-45-10 +0000,nypost,"Mexican resort town, airport under siege after...",https://nypost.com/2026/02/22/world-news/mexic...
74,31,2026-02-22-16-10-08 +0000,bbc,USA win men's Olympic ice hockey gold for firs...,https://www.bbc.com/sport/ice-hockey/articles/...
165,30,2026-02-22-00-33-48 +0000,nypost,Behind shadowy union boss pushing California’s...,https://nypost.com/2026/02/21/us-news/dave-reg...
152,30,2026-02-22-03-17-28 +0000,cbc,"Greenland PM declines Trump's hospital ship, u...",https://www.cbc.ca/news/world/trump-hospital-s...
64,30,2026-02-22-17-17-27 +0000,nypost,C-SPAN caller using Trump’s pseudonym and rema...,https://nypost.com/2026/02/22/us-news/c-span-c...
168,29,2026-02-22-00-15-48 +0000,nypost,"Senior MP demands UK probe ex-Prince Andrew, P...",https://nypost.com/2026/02/21/world-news/senio...
30,26,2026-02-22-20-01-36 +0000,nyt,Homeland Security to Shut TSA PreCheck and Glo...,https://www.nytimes.com/2026/02/22/us/tsa-prec...
59,26,2026-02-22-17-46-59 +0000,nypost,Mayor Mamdani announces ‘full classic snow day...,https://nypost.com/2026/02/22/us-news/mayor-ma...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
